In [1]:
from tasks.building import BuildingSegmentationDatasetconda 


ImportError: /home/sl636/miniconda3/envs/geospatial/lib/python3.10/site-packages/../../././libgnutls.so.30: undefined symbol: mpn_copyi, version HOGWEED_6

In [ ]:
def _load_building_data(normalization, augmentations):
    # Paths to train and test set (as split from INRIA)
    train_imgs_path = "/scratch/saad/1000_images/"
    train_masks_path = "/scratch/saad/1000_masks/"
    val_imgs_path = "/scratch/saad/1000_val_images/"
    val_masks_path = "/scratch/saad/1000_val_masks/"

    train_imgs = os.listdir(train_imgs_path)
    val_imgs = os.listdir(val_imgs_path)

    logging.debug(
        f"We are using {len(train_imgs)} training images and {len(val_imgs)} validation images from the INRIA building dataset.")

    # We want to ensure that the normalization scheme is considered. In the case
    # that we are using a pretrained method, it might be better to use that
    # pretrained normalization mean and standard deviation. Otherwise, it would
    # be better to use the normalization scheme applied on the original dataset.
    # By specifying 'data', we are saying that we want to use the calculated
    # mean and standard deviation on the dataset. Other normalization methods
    # should be added as more pre-trained methods are supported.
    if normalization == 'data':
        # This normalization was calculated by taking several sample
        # images (as tensors) and calculating the average RGB value along with the
        # standard deviation.
        print("Normalizing using the data.")
        normalize = {
            'mean': [0.406, 0.428, 0.394],
            'std': [0.201, 0.183, 0.176]
        }
    elif normalization == 'imagenet':
        print("Normalize using imagenet.")
        # This normalization scheme uses the means and weights for ImageNet.
        normalize = {
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225]
        }
    else:
        raise NotImplementedError("This normalization scheme isn't supported.")

    # Include everything but the kitchen sink...
    # we want a large collection of transformations.
    # The first four ensure that each image is represented
    # in 8 separate ways...the next 3 transformations
    # affect the coloration and pixel values, whereas
    # the final 2 ensure that the image is in a format
    # that the model likes.
    
    tr_normalize = transforms.Normalize(
        mean=normalize['mean'], std=normalize['std']
    )

    train_transform = transforms.Compose(
        [
            transforms.ToTensor(),
            tr_normalize
        ]
    )

    test_transform = transforms.Compose(
        [transforms.ToTensor(), tr_normalize]
    )

    # Load the training dataset
    logging.debug("Creating the training dataset.")
    if augmentations:
        train_dataset = BuildingSegmentationDataset(
            train_imgs_path, train_imgs, train_masks_path,
            transform=train_transform, augmentations=aug
        )
    else:
        train_dataset = BuildingSegmentationDataset(
            train_imgs_path, train_imgs, train_masks_path,
            transform=train_transform
        )
    # Load the test dataset
    logging.debug("Creating the test dataset.")
    test_dataset = BuildingSegmentationDataset(
        val_imgs_path, val_imgs, val_masks_path,
        transform=test_transform
    )
    # Return the training and test dataset
    return train_dataset, test_dataset
